**1. Подключаем библиотеки**

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import sklearn as sk
import skimage


**2.Получаем доступ к данным** Т.к. изображения рисинок уже рассортированы по папкам с названиями видов, то отсюда информацию о названиях видов и возьмём

In [2]:
Path = '/kaggle/input/rice-image-dataset/Rice_Image_Dataset'
Species=os.listdir(Path)
del(Species[3])#игнорируем текстовый файл, он чисто описательный
print("Виды турецкого риса: "+', '.join(Species))

**3.Загружем данные** С целью ускорения процесса, масштабируем изображения и делаем их полностью серыми (уходим от RGB-формата). Для работы нейросети производим спрямление массивов

In [3]:
rescale = skimage.transform.rescale
imread = skimage.io.imread
Data = {'Image':[], 'Species':[]}

for categoty in Species:
    m=0
    for string in os.listdir(Path+'/'+categoty):
        image = rescale(imread(Path+'/'+categoty+'/'+string), 0.3, anti_aliasing=False)
        Data['Image'].append(list(image.reshape(image.shape[0]**2)))
        Data['Species'].append(categoty)
        
        if m==0:
            print("Работаем с "+categoty)
        m+=1
        if m>9999:
            break
print("Конец загрузки изображений")
Data = pd.DataFrame(Data).sample(frac=1)

In [8]:
Data

Демонстрируем возможностб частично восстановить изображение
Также проводим разбиени данных на обучающую и тестовую выборку

In [9]:
from sklearn.model_selection import train_test_split
f=np.array(Data['Image'][0])
skimage.io.imshow(f.reshape((round(np.sqrt(f.shape[0])),round(np.sqrt(f.shape[0])),1)))
print('Рисинка')
X_train, X_test, y_train, y_test = train_test_split(Data['Image'],Data['Species'],random_state=0)

Подключаем нейросеть, используем метод близжайшего соседа
Преобразуем данные в формат, с которым классификатор согласен работать

In [10]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
X_train_1,y_train_1=np.array(list(X_train)),list(y_train)
knn.fit(X_train_1,y_train_1)

Проверяем работоспособность нашей модели

In [7]:
X_test_1 = np.array(list(X_test))
Prediction = knn.predict(X_test_1)

from sklearn.metrics import accuracy_score

print('Доля правильных ответов - ',100*accuracy_score(y_test, Prediction),'%')
from sklearn.metrics import confusion_matrix

mat = confusion_matrix(y_test, Prediction)


fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111)
ax.matshow(mat)
for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
        ax.text(x=j, y=i,s=mat[i, j], va='center', ha='center', size='xx-large', c = 'r')
ax.set_xticks([0,1,2,3,4])
ax.set_yticks([0,1,2,3,4])
ax.set_yticklabels(Species)
ax.set_xticklabels(Species)